# Dictionary compilation from Για να κ̇οντούμε γρούσσα νάμου

This notebook aims to create a dictionary from the words contained in Ioannis Kambysis' book, _Για να κχοντούμε γρούσσα νάμου_ (2020) using Google Lens OCR.

# Preparation

In [1]:
# Data wrangling
import pandas as pd
import numpy as np

# Others
import re
import pyperclip as pc
import unicodedata
import copy
import os

## Auxiliary functions

In [2]:
def remove_accents(input_text):
    nfkd_form = unicodedata.normalize('NFKD', input_text)
    return ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

In [3]:
def compute_masculine_paradigm(entry_: pd.Series):
    """
    Computes the hypothesized paradigm for a given masculine entry.
    """

    # Remove accents from the  plural
    entry_['plural'] = remove_accents(entry_['plural'])

    ### Apply rules ###
    # A1: plural in -οι
    if entry_['plural'].endswith('οι'):
        return "Α1"
    
    # A4: plural in -νε 
    # NOTE: plurals are formed by adding -ούνε to the singular
    # but it is checked here as -νε for simplicity
    elif entry_['plural'].endswith('ουνε'):
        return "Α4"
    
    # A5: plural in -δε
    elif entry_['plural'].endswith('δε'):
        return "Α5"
    
    # A2: plural in -ε
    elif entry_['plural'].endswith('ε'):
        return "Α2"
    
    # A3: plural in -ου
    elif entry_['plural'].endswith('ου'):
        return "Α3"
    
    # NaN if no rule applies
    else:
        return "Α"

In [4]:
def compute_femenine_paradigm(entry_: pd.Series):
    """
    Computes the hypothesized paradigm for a given feminine entry.
    """

    # Remove accents from the word
    entry_['singular'] = remove_accents(entry_['singular'])

    ### Apply rules ###
    # Θ3: plural in -άε
    # If it does not hold, remove the accent from the plural suffix
    if entry_['plural'].endswith('άε'):
        return "Θ3"
    else:
        entry_['plural'] = remove_accents(entry_['plural'])

    # Θ2: plural in -λε
    if entry_['plural'].endswith('λε'):
        return "Θ2"    
    
    # Θ4: plural in -δε and word does not end in -ου
    elif entry_['plural'].endswith('δε') and not entry_['singular'].endswith('ου'):
        return "Θ4"

    # Θ5: plural in -δε and word ends in -ου
    elif entry_['plural'].endswith('δε') and entry_['singular'].endswith('ου'):
        return "Θ5"
    
    # Θ1: rest of plurals in ending in -ε
    elif entry_['plural'].endswith('ε'):
        return "Θ1"
        
    # NaN if no rule applies
    else:
        return "Θ"

In [5]:
def compute_neuter_paradigm(entry_):
    """
    Computes the hypothesized paradigm for a given neuter entry.
    """

    # Remove accents from the word and plural
    for key in ['singular', 'plural']:
        if entry_[key] is not np.nan:
            entry_[key] = remove_accents(entry_[key])    

    ### Apply rules ###
    # Υ2: word ends in -μα
    if entry_['singular'].endswith('μα'):
        return "Υ2"
    
    # Υ3: plural ends in -ια
    elif entry_['plural'].endswith('ια'):
        return "Υ3"
    
    # Υ4: word ends in -ι and plural ends in -τα
    elif entry_['singular'].endswith('ι') and entry_['plural'].endswith('ητα'):
        return "Υ4"
    
    # Υ5: word ends in -ε and plural ends in -τα
    elif entry_['singular'].endswith('ε') and entry_['plural'].endswith('ατα'):
        return "Υ5"
    
    # Y1: rest of plurals in -α
    elif entry_['plural'].endswith('α'):
        return "Υ1"
    
    # Υ if no rule applies
    else:
        return 'Υ'

In [6]:
def compute_paradigms(entry: pd.Series):
    """
    Extracts noun paradigms based on the plural suffixes.
    """

    # If the plural key is NaN, return the gender
    if entry['plural'] is np.nan:
        return entry['gender']

    # Create a modifyable copy of the df_dict
    # The values will lose the accents, which is not desired
    # for the final dataframe
    copy_entry = copy.deepcopy(entry)

    # Masculine nouns
    if entry['article'] == 'Ο':
        paradigm = compute_masculine_paradigm(copy_entry)
    
    # Femenine nouns
    elif entry['article'] == 'Α':
        paradigm = compute_femenine_paradigm(copy_entry)

    # Neuter nouns
    elif entry['article'] == 'Το':
        paradigm = compute_neuter_paradigm(copy_entry)

    # NaN if no rule applies
    else:
        paradigm = np.nan

    return paradigm

# Set clean text

In [38]:
page = 47
text = """
ένα, νία, ένα ένας, μία, ένα
δύου, δύ (ρ) δύο
τσει(ρ), τσει(ρ), τσία τρεις, τρεις, τρία
τέσσερι(ρ), τέσσερι(ρ), τέσσερα τέσσερις, τέσσερις, τέσσερα
πέντε(ρ) πέντε
έξε(ρ) έξι
εφτά(ρ) εφτά
οχτώ (ρ) οχτώ
ενία(ρ) εννέα δέκα(ρ) δέκα
έντεκα(ρ) έντεκα
δώδεκα(ρ) δώδεκα
δεκατσεί(ρ), δεκατσεί(ρ), δεκαταία δεκατρείς, δεκατρείς
, δεκατρία
δεκατέσσερι(ρ), δεκατέσσερι(ρ), δεκατέσσερα δεκατέσσερις, δεκατέσσερις, δεκατέσσερα
δεκαπέντε(ρ) δεκαπέντε
δεκαέξι(ρ) δεκαέξι
δεκαεφτά(ρ) δεκαεφτά δεκαοχτώ (ρ) δεκαοχτώ
δεκαενία(ρ) δεκαεννέα
είκοσι (ρ) είκοσι
τριάντα (ρ) τριάντα
εκατό (ρ) εκατό
(ε)διακόσοι(ρ), (ε)διακόσοι(ρ), (ε)διακόσα διακόσιοι -ες -α (ε)τρακόσοι(ρ), (ε)τρακόσοι(ρ), (ε)τρακόσα τρακόσιοι -ες να
χίλιοι(ρ), χίλιοι(ρ), χίλια χίλιοι χίλιες χίλια ένα μελεούνι για μεγάλο πλήθος
ο πρώτε ο πρώτος, α πρώτε η πρώτη, το πρώκιου το πρώτο ο δεύτερε ο δεύτερος, α δεύτερε η δεύτερη, το δεύτερ(ζ)ιου το
00
είτε ο τρίτος - τριζίτε η τρίτη, το τρ(ζ)ίτε (τρ(ζ)ίκιου) το
"""

In [39]:
# Save cleaned text
with open(rf'./cleaned_pages/page_{page}.txt', 'w', encoding='utf-8') as f:
    f.write(text.strip())

In [40]:
# Separate Tsakonian and Greek parts
text_list = text.strip().split('\n')
tsakonian_text = [line.split(' ; ')[0] for line in text_list]
greek_text = [line.split(' ; ')[1] for line in text_list]

assert len(tsakonian_text) == len(greek_text), 'Tsakonian and Greek parts must have the same length.'

In [41]:
# Build initial dataframe
temp_df = pd.DataFrame({'tsakonian' : tsakonian_text, 
                        'greek' : greek_text})
temp_df

,tsakonian,greek
0,Ο κούβελε - Οι κουβέλοι,Το σκαφίδι - Τα σκαφίδια
1,Ο ακ̇ό - Οι ακού,Το ασκί - Τα ασκιά
2,Το σκόπουλε - Τα σκόπουα,Το τουλούμι - Τα τουλούμια
3,Το παστσήρι,Το πλαστήρι
4,Το π̇ιτόκαλε - Τα πιτόκα,Ο πλάστης - Οι πλάστες
5,Α κράκα - Οι κράτσε,Το κλειδί - Τα κλειδιά
6,Ο λύχινε - Οι λύχινοι,Το λυχνάρι - Τα λυχνάρια
7,Ο κρεβάτ̇α - Οι κρεβάτοι,Το κρεβάτι - Τα κρεβάτια
8,Το κόγκισμα - Τα κογκίσματα,Το εικόνισμα - Τα εικονίσματα
9,Το καγκήλι - Τα καγκήλια,Το καντήλι - Τα καντήλια


## Process Tsakonian column

### Determine gender

In [42]:
# Extract the gender
gender_dict = {
    "Ο" : "Α",
    "Α" : "Θ",
    "To" : "Υ",
    "Το" : "Υ",
    "Τα" : "Υ",
    "Οι" : np.nan,
}

# Create columns
temp_df['article'] = temp_df['tsakonian'].apply(lambda x: x.split(' ')[0])
temp_df['gender'] = temp_df['article'].apply(lambda x: gender_dict[x] if x in gender_dict.keys() else np.nan)
temp_df

,tsakonian,greek,article,gender
0,Ο κούβελε - Οι κουβέλοι,Το σκαφίδι - Τα σκαφίδια,Ο,Α
1,Ο ακ̇ό - Οι ακού,Το ασκί - Τα ασκιά,Ο,Α
2,Το σκόπουλε - Τα σκόπουα,Το τουλούμι - Τα τουλούμια,Το,Υ
3,Το παστσήρι,Το πλαστήρι,Το,Υ
4,Το π̇ιτόκαλε - Τα πιτόκα,Ο πλάστης - Οι πλάστες,Το,Υ
5,Α κράκα - Οι κράτσε,Το κλειδί - Τα κλειδιά,Α,Θ
6,Ο λύχινε - Οι λύχινοι,Το λυχνάρι - Τα λυχνάρια,Ο,Α
7,Ο κρεβάτ̇α - Οι κρεβάτοι,Το κρεβάτι - Τα κρεβάτια,Ο,Α
8,Το κόγκισμα - Τα κογκίσματα,Το εικόνισμα - Τα εικονίσματα,Το,Υ
9,Το καγκήλι - Τα καγκήλια,Το καντήλι - Τα καντήλια,Το,Υ


### Determine plural paradigm

In [43]:
# Extract singular and plurals
temp_df['singular'] = temp_df['tsakonian'].apply(lambda x: x.split(' - ')[0].split(' ')[1])
temp_df['plural'] = temp_df['tsakonian'].apply(lambda x: x.split(' - ')[1].split(' ')[1] if ' - ' in x else np.nan)

temp_df

,tsakonian,greek,article,gender,singular,plural
0,Ο κούβελε - Οι κουβέλοι,Το σκαφίδι - Τα σκαφίδια,Ο,Α,κούβελε,κουβέλοι
1,Ο ακ̇ό - Οι ακού,Το ασκί - Τα ασκιά,Ο,Α,ακ̇ό,ακού
2,Το σκόπουλε - Τα σκόπουα,Το τουλούμι - Τα τουλούμια,Το,Υ,σκόπουλε,σκόπουα
3,Το παστσήρι,Το πλαστήρι,Το,Υ,παστσήρι,NaN
4,Το π̇ιτόκαλε - Τα πιτόκα,Ο πλάστης - Οι πλάστες,Το,Υ,π̇ιτόκαλε,πιτόκα
5,Α κράκα - Οι κράτσε,Το κλειδί - Τα κλειδιά,Α,Θ,κράκα,κράτσε
6,Ο λύχινε - Οι λύχινοι,Το λυχνάρι - Τα λυχνάρια,Ο,Α,λύχινε,λύχινοι
7,Ο κρεβάτ̇α - Οι κρεβάτοι,Το κρεβάτι - Τα κρεβάτια,Ο,Α,κρεβάτ̇α,κρεβάτοι
8,Το κόγκισμα - Τα κογκίσματα,Το εικόνισμα - Τα εικονίσματα,Το,Υ,κόγκισμα,κογκίσματα
9,Το καγκήλι - Τα καγκήλια,Το καντήλι - Τα καντήλια,Το,Υ,καγκήλι,καγκήλια


In [44]:
# Compute paradigms
temp_df['paradigm'] = temp_df.apply(compute_paradigms, axis=1)
temp_df

,tsakonian,greek,article,gender,singular,plural,paradigm
0,Ο κούβελε - Οι κουβέλοι,Το σκαφίδι - Τα σκαφίδια,Ο,Α,κούβελε,κουβέλοι,Α1
1,Ο ακ̇ό - Οι ακού,Το ασκί - Τα ασκιά,Ο,Α,ακ̇ό,ακού,Α3
2,Το σκόπουλε - Τα σκόπουα,Το τουλούμι - Τα τουλούμια,Το,Υ,σκόπουλε,σκόπουα,Υ1
3,Το παστσήρι,Το πλαστήρι,Το,Υ,παστσήρι,NaN,Υ
4,Το π̇ιτόκαλε - Τα πιτόκα,Ο πλάστης - Οι πλάστες,Το,Υ,π̇ιτόκαλε,πιτόκα,Υ1
5,Α κράκα - Οι κράτσε,Το κλειδί - Τα κλειδιά,Α,Θ,κράκα,κράτσε,Θ1
6,Ο λύχινε - Οι λύχινοι,Το λυχνάρι - Τα λυχνάρια,Ο,Α,λύχινε,λύχινοι,Α1
7,Ο κρεβάτ̇α - Οι κρεβάτοι,Το κρεβάτι - Τα κρεβάτια,Ο,Α,κρεβάτ̇α,κρεβάτοι,Α1
8,Το κόγκισμα - Τα κογκίσματα,Το εικόνισμα - Τα εικονίσματα,Το,Υ,κόγκισμα,κογκίσματα,Υ2
9,Το καγκήλι - Τα καγκήλια,Το καντήλι - Τα καντήλια,Το,Υ,καγκήλι,καγκήλια,Υ3


## Process Greek column

In [45]:
# Keep only the singular form of the word for Greek
temp_df['greek'] = temp_df['greek'].apply(lambda x: x.split(' - ')[0])

# If the word starts with an article, remove it
articles = ['Ο ', 'Η ', 'Το ', 'Οι ', 'Τα ']
temp_df['greek'] = temp_df['greek'].apply(lambda x: ' '.join(x.split(' ')[1:]) if x.startswith(tuple(articles)) else x)

# Lowercase
temp_df['greek'] = temp_df['greek'].apply(lambda x: x.lower())

temp_df

,tsakonian,greek,article,gender,singular,plural,paradigm
0,Ο κούβελε - Οι κουβέλοι,σκαφίδι,Ο,Α,κούβελε,κουβέλοι,Α1
1,Ο ακ̇ό - Οι ακού,ασκί,Ο,Α,ακ̇ό,ακού,Α3
2,Το σκόπουλε - Τα σκόπουα,τουλούμι,Το,Υ,σκόπουλε,σκόπουα,Υ1
3,Το παστσήρι,πλαστήρι,Το,Υ,παστσήρι,NaN,Υ
4,Το π̇ιτόκαλε - Τα πιτόκα,πλάστης,Το,Υ,π̇ιτόκαλε,πιτόκα,Υ1
5,Α κράκα - Οι κράτσε,κλειδί,Α,Θ,κράκα,κράτσε,Θ1
6,Ο λύχινε - Οι λύχινοι,λυχνάρι,Ο,Α,λύχινε,λύχινοι,Α1
7,Ο κρεβάτ̇α - Οι κρεβάτοι,κρεβάτι,Ο,Α,κρεβάτ̇α,κρεβάτοι,Α1
8,Το κόγκισμα - Τα κογκίσματα,εικόνισμα,Το,Υ,κόγκισμα,κογκίσματα,Υ2
9,Το καγκήλι - Τα καγκήλια,καντήλι,Το,Υ,καγκήλι,καγκήλια,Υ3


## Final processing

In [46]:
# Extract final table
columns = ['singular', 'greek', 'paradigm']
final_table = temp_df[columns].copy()

# Add a 'source' column with value 1
final_table['source'] = 1

final_table

,singular,greek,paradigm,source
0,κούβελε,σκαφίδι,Α1,1
1,ακ̇ό,ασκί,Α3,1
2,σκόπουλε,τουλούμι,Υ1,1
3,παστσήρι,πλαστήρι,Υ,1
4,π̇ιτόκαλε,πλάστης,Υ1,1
5,κράκα,κλειδί,Θ1,1
6,λύχινε,λυχνάρι,Α1,1
7,κρεβάτ̇α,κρεβάτι,Α1,1
8,κόγκισμα,εικόνισμα,Υ2,1
9,καγκήλι,καντήλι,Υ3,1


In [48]:
# Save the dataframe to an excel
path_ = f'./results/{page}.xlsx'
final_table.to_excel(path_, index=False)
print(f'{page} saved.')

# Open the excel
complete_path = os.path.abspath(path_)
os.startfile(complete_path)

35 saved.
